In [1]:
import time
import numpy as np

import os
from datetime import date
from scipy.constants import elementary_charge, proton_mass 

import visit
visit.Launch()

True

In [2]:
database_prefix = '/Users/vonderlinden2/rsx_analysis/writing_to_vtk/output/2016-08-14-17-44'

# Define functions

In [3]:
def define_expressions(alpha=8.1e5):
    visit.DefineVectorExpression("B", "{B_x, B_y, B_z}")
    visit.DefineVectorExpression("B_norm", "{B_norm_x, B_norm_y, B_norm_z}")
    visit.DefineVectorExpression("B_perp", "{B_x, B_y, 0}")
    visit.DefineVectorExpression("B_para", "{0, 0, B_z}")
    visit.DefineScalarExpression("B_para_scalar", "B_z")
    
    visit.DefineVectorExpression("J", "{j_x, j_y, j_z}")
    visit.DefineScalarExpression("J_mag", "sqrt(j_x^2 + j_y^2 + j_z^2)")
    visit.DefineVectorExpression("J_perp", "{j_x, j_y, 0}")
    visit.DefineVectorExpression("J_para", "{0, 0, j_z}")
    visit.DefineScalarExpression("J_para_scalar", "j_z")
    
    visit.DefineVectorExpression("u_e_norm", "{u_e_norm_x, u_e_norm_y, u_e_norm_z}")
    visit.DefineVectorExpression("u_e", "%1.2e * u_e_norm" % alpha)
    visit.DefineVectorExpression("u_e_perp", "{dot(u_e, {1, 0, 0}), dot(u_e, {0, 1, 0}), 0}")
    visit.DefineScalarExpression("u_e_para_scalar", "dot(u_e, {0, 0, 1})")
    
    
    visit.DefineVectorExpression("u_i_term1", "{u_i_term1_x, u_i_term1_y, u_i_term1_z}")
    visit.DefineVectorExpression("u_i_term1_perp", "{u_i_term1_x, u_i_term1_y, 0}")
    visit.DefineVectorExpression("u_i_term1_para", "{0, 0, u_i_term1_z}") 
    visit.DefineScalarExpression("u_i_term1_para_scalar", "u_i_term1_z") 
    
    visit.DefineVectorExpression("omega_i_term1", "{w_i_term1_x, w_i_term1_y, w_i_term1_z}")
    visit.DefineVectorExpression("omega_i_term1_perp", "{w_i_term1_x, w_i_term1_y, 0}")
    visit.DefineVectorExpression("omega_i_term1_para", "{0, 0, w_i_term1_z}")
    visit.DefineScalarExpression("omega_i_term1_para_scalar", "w_i_term1_z")
    
    visit.DefineVectorExpression("omega_i_term2", "{w_i_term2_x, w_i_term2_y, w_i_term2_z} * %1.2e" % alpha)
    visit.DefineVectorExpression("omega_i_term2_perp", "{w_i_term2_x, w_i_term2_y, 0} * %1.2e" % alpha)
    visit.DefineVectorExpression("omega_i_term2_para", "{0, 0, w_i_term2_z} * %1.2e" % alpha)
    visit.DefineScalarExpression("omega_i_term2_para_scalar", "w_i_term2_z * %1.2e" % alpha)
    
    visit.DefineVectorExpression("u_i", "u_i_term1 + u_e")
    visit.DefineVectorExpression("u_i_perp", "{dot(u_i, {1, 0, 0}), dot(u_i, {0, 1, 0}), 0}")
    visit.DefineVectorExpression("u_i_para", "{0, 0, dot(u_i, {0, 0, 1})}")
    visit.DefineScalarExpression("u_i_para_scalar", "dot(u_i, {0, 0, 1})")
    
    visit.DefineVectorExpression("omega_i", "omega_i_term1 + omega_i_term2 * %1.2e" % alpha)
    visit.DefineVectorExpression("omega_i_perp", "{dot(omega_i, {1, 0, 0}), dot(omega_i, {0, 1, 0}), 0}")
    visit.DefineVectorExpression("omega_i_para", "{0, 0, dot(omega_i, {0, 0, 1})}")
    visit.DefineScalarExpression("omega_i_para_scalar", "dot(omega_i, {0, 0, 1})")
    
    visit.DefineVectorExpression("Omega_i", "B +" + str(proton_mass) + "/" + str(elementary_charge) + "*omega_i")
    visit.DefineVectorExpression("Omega_i_perp", "{dot(Omega_i, {1, 0, 0}), dot(Omega_i, {0, 1, 0}), 0}")
    visit.DefineVectorExpression("Omega_i_para", "{0, 0, dot(Omega_i, {0, 0, 1})}")
    visit.DefineScalarExpression("Omega_i_para_scalar", "dot(Omega_i, {0, 0, 1})")

In [4]:
def launch_points(center):
    r"""
    """
    thetas = np.linspace(0, 3./4.*np.pi, 60)
    thetas = np.concatenate((thetas, np.linspace(5./4.*np.pi, 2.*np.pi, 60)))
    x_outer = 0.01 * np.cos(thetas) + center[0]
    y_outer = 0.01 * np.sin(thetas) + center[1]
    z_outer = 0.249 * np.ones((x_outer.size))
    points_outer = np.empty((x_outer.size + y_outer.size + z_outer.size))
    points_outer[0::3] = x_outer
    points_outer[1::3] = y_outer
    points_outer[2::3] = z_outer
    points_outer = tuple(points_outer)

    thetas = np.linspace(0, 2.*np.pi, 80)
    x_inner = 0.005 * np.cos(thetas) + center[0]
    y_inner = 0.005 * np.sin(thetas) + center[1]
    z_inner = 0.249 * np.ones(x_inner.size)
    points_inner = np.empty((x_inner.size + y_inner.size + z_inner.size))
    points_inner[0::3] = x_inner
    points_inner[1::3] = y_inner 
    points_inner[2::3] = z_inner
    points_inner = tuple(points_inner)
    
    return points_outer, points_inner

In [5]:
def setup_current_pseudocolor(colortable="Greens"):
    r"""
    """
    visit.AddPlot("Pseudocolor", "J_mag", 1, 0)
    PseudocolorAtts = visit.PseudocolorAttributes()
    PseudocolorAtts.scaling = PseudocolorAtts.Linear  
    PseudocolorAtts.limitsMode = PseudocolorAtts.OriginalData 
    PseudocolorAtts.colorTableName = colortable
    visit.SetPlotOptions(PseudocolorAtts)
    visit.AddOperator("Slice", 0)
    SliceAtts = visit.SliceAttributes()
    SliceAtts.originType = SliceAtts.Intercept 
    SliceAtts.originIntercept = 0.249
    SliceAtts.axisType = SliceAtts.ZAxis
    SliceAtts.project2d = 0
    visit.SetOperatorOptions(SliceAtts, 0)
    return PseudocolorAtts, SliceAtts

In [6]:
def setup_massless_electron_canonical_flux_tubes(points_outer, points_inner):
    r"""
    """
    visit.AddPlot("Streamline", "B", 1, 0)
    StreamlineAtts_outer = visit.StreamlineAttributes()
    StreamlineAtts_outer.sourceType = StreamlineAtts_outer.SpecifiedPointList
    StreamlineAtts_outer.SetPointList(points_outer)
    StreamlineAtts_outer.coloringMethod = StreamlineAtts_outer.Solid  
    StreamlineAtts_outer.colorTableName = "Default"
    StreamlineAtts_outer.singleColor = (255, 0, 0, 255)
    StreamlineAtts_outer.showHeads = 1
    StreamlineAtts_outer.headRadiusBBox = 0.015
    StreamlineAtts_outer.legendFlag = 0
    visit.SetPlotOptions(StreamlineAtts_outer)

    visit.AddPlot("Streamline", "B", 1, 0)
    StreamlineAtts_inner = visit.StreamlineAttributes()
    StreamlineAtts_inner.sourceType = StreamlineAtts_inner.SpecifiedPointList
    StreamlineAtts_inner.SetPointList(points_inner)
    StreamlineAtts_inner.coloringMethod = StreamlineAtts_inner.Solid
    StreamlineAtts_inner.colorTableName = "Default"
    StreamlineAtts_inner.singleColor = (190, 64, 0, 255)
    StreamlineAtts_inner.showHeads = 1
    StreamlineAtts_inner.headRadiusBBox = 0.015
    StreamlineAtts_inner.legendFlag = 0
    visit.SetPlotOptions(StreamlineAtts_inner)
    
    return StreamlineAtts_outer, StreamlineAtts_inner

In [7]:
def setup_field_line(center=(0.01, 0.01, 0.249), outer_radius=0.01):
    visit.AddPlot("Streamline", "B", 1, 0)
    StreamlineAtts_line = visit.StreamlineAttributes()
    StreamlineAtts_line.sourceType = StreamlineAtts_line.SpecifiedPoint
    StreamlineAtts_line.pointSource = (center[0], center[1] + outer_radius, center[2])
    StreamlineAtts_line.coloringMethod = StreamlineAtts_line.Solid
    StreamlineAtts_line.singleColor = (255, 255, 153, 255)
    StreamlineAtts_line.legendFlag = 0
    StreamlineAtts_line.showSeeds = 0
    visit.SetPlotOptions(StreamlineAtts_line)
    
    return StreamlineAtts_line

In [8]:
def setup_axes():
    AnnotationAtts = visit.AnnotationAttributes()
    AnnotationAtts.axes3D.autoSetScaling = 0
    AnnotationAtts.axes3D.xAxis.title.visible = 0
    AnnotationAtts.axes3D.yAxis.title.visible = 0
    AnnotationAtts.axes3D.zAxis.title.visible = 0
    AnnotationAtts.axes3D.xAxis.label.font.scale = 3
    AnnotationAtts.axes3D.xAxis.label.scaling = -2
    AnnotationAtts.axes3D.yAxis.label.font.scale = 3
    AnnotationAtts.axes3D.yAxis.label.scaling = -2
    AnnotationAtts.axes3D.zAxis.label.font.scale = 3
    AnnotationAtts.axes3D.zAxis.label.scaling = -2
    visit.SetAnnotationAttributes(AnnotationAtts)
    return AnnotationAtts

In [9]:
today = str(date.today())
out_dir = '../output/' + today
try:
    os.makedirs(out_dir)
except:
    pass

output_path = out_dir + '/canonical_electron_fluxtube_'

define_expressions()
visit.OpenDatabase(database_prefix + '/Bdot_triple_probe_quantities*.vtk database')
field_nulls = np.loadtxt('../../centroid_fitting/output/2016-08-12/field_nulls.txt')
PseudocolorAtts, SliceAtts = setup_current_pseudocolor()
points_outer, points_inner = launch_points(field_nulls[0])
AnnotationAtts = setup_axes()
StreamlineAtts_line = setup_field_line(np.concatenate((field_nulls[0], [0.249])), outer_radius=0.01)


(electron_StreamlineAtts_outer, 
 electron_StreamlineAtts_inner) = setup_massless_electron_canonical_flux_tubes(points_outer, points_inner)
visit.DrawPlots()

1L

In [10]:
print visit.GetView3D()

viewNormal = (0, 0, 1)
focus = (0.00202222, 0.000976744, 0.331997)
viewUp = (0, 1, 0)
viewAngle = 30
parallelScale = 0.088383
nearPlane = -0.176766
farPlane = 0.176766
imagePan = (0, 0)
imageZoom = 1
perspective = 1
eyeAngle = 2
centerOfRotationSet = 0
centerOfRotation = (0.00202222, 0.000976744, 0.331997)
axis3DScaleFlag = 0
axis3DScales = (1, 1, 1)
shear = (0, 0, 1)
windowValid = 1



In [11]:
view = visit.GetView3D()
view.SetViewNormal((-0.731293, 0.40847, 0.546227))
view.SetFocus((0.00202222, 0.000976744, 0.331997))
view.SetViewUp((0.322268, 0.91274, -0.251095))
view.SetViewAngle(30)
view.SetParallelScale(0.088383)
view.SetNearPlane(-0.176766)
view.SetImagePan((0, 0))
view.SetImageZoom(1)
view.SetPerspective(1)
view.SetEyeAngle(2)
view.SetCenterOfRotationSet(0)
view.SetCenterOfRotation((0.00202222, 0.000976744, 0.331997))
view.SetAxis3DScaleFlag(0)
view.SetAxis3DScales((1,1,1))
view.SetShear((0,0,1))
view.SetWindowValid(0)
visit.SetView3D(view)

1L

In [12]:
visit.OpenGUI()

1L

In [15]:
save_atts = visit.SaveWindowAttributes()
save_atts.format = save_atts.PNG
save_atts.height = 1080
save_atts.width = 1920
save_atts.family = 0
visit.SetSaveWindowAttributes(save_atts)
ending='.png'

for time_point in xrange(0, 206):
    print time_point
    save_atts.fileName = output_path + str(time_point).zfill(4) + ending
    visit.SetSaveWindowAttributes(save_atts)
    
    points_outer, points_inner = launch_points(field_nulls[time_point])
    
    visit.SetActivePlots(1)
    StreamlineAtts_line.pointSource = (field_nulls[time_point][0], field_nulls[time_point][1] + 0.01, 0.249)
    visit.SetPlotOptions(StreamlineAtts_line)
    
    visit.SetActivePlots(2)
    electron_StreamlineAtts_outer.SetPointList(points_outer)
    visit.SetPlotOptions(electron_StreamlineAtts_outer)
    
    visit.SetActivePlots(3)
    electron_StreamlineAtts_inner.SetPointList(points_inner)
    visit.SetPlotOptions(electron_StreamlineAtts_inner)
    
    visit.SetTimeSliderState(time_point)
    
    name = visit.SaveWindow()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205


In [16]:
save_atts = visit.SaveWindowAttributes()
save_atts.format = save_atts.PNG
save_atts.height = 1080
save_atts.width = 1920
save_atts.family = 0
visit.SetSaveWindowAttributes(save_atts)
ending='.png'

for time_point in xrange(226, 250):
    print time_point
    save_atts.fileName = output_path + str(time_point - 20).zfill(4) + ending
    visit.SetSaveWindowAttributes(save_atts)
    
    points_outer, points_inner = launch_points(field_nulls[time_point])
    
    visit.SetActivePlots(1)
    StreamlineAtts_line.pointSource = (field_nulls[time_point][0], field_nulls[time_point][1] + 0.01, 0.249)
    visit.SetPlotOptions(StreamlineAtts_line)
    
    visit.SetActivePlots(2)
    electron_StreamlineAtts_outer.SetPointList(points_outer)
    visit.SetPlotOptions(electron_StreamlineAtts_outer)
    
    visit.SetActivePlots(3)
    electron_StreamlineAtts_inner.SetPointList(points_inner)
    visit.SetPlotOptions(electron_StreamlineAtts_inner)
    
    visit.SetTimeSliderState(time_point)
    
    name = visit.SaveWindow()

226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249


In [15]:
save_atts = visit.SaveWindowAttributes()
save_atts.format = save_atts.PNG
save_atts.height = 1080
save_atts.width = 1920
save_atts.family = 0
visit.SetSaveWindowAttributes(save_atts)
ending='.png'

for time_point in xrange(visit.TimeSliderGetNStates()):
    print time_point
    save_atts.fileName = output_path + str(time_point).zfill(4) + ending
    visit.SetSaveWindowAttributes(save_atts)
    
    points_outer, points_inner = launch_points(field_nulls[time_point])
    
    visit.SetActivePlots(1)
    StreamlineAtts_line.pointSource = (field_nulls[time_point][0], field_nulls[time_point][1] + 0.01, 0.249)
    visit.SetPlotOptions(StreamlineAtts_line)
    
    visit.SetActivePlots(2)
    electron_StreamlineAtts_outer.SetPointList(points_outer)
    visit.SetPlotOptions(electron_StreamlineAtts_outer)
    
    visit.SetActivePlots(3)
    electron_StreamlineAtts_inner.SetPointList(points_inner)
    visit.SetPlotOptions(electron_StreamlineAtts_inner)
    
    visit.SetTimeSliderState(time_point)
    
    name = visit.SaveWindow()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249


In [10]:
visit.OpenGUI()

1L

In [17]:
!ffmpeg -framerate 30 -i ../output/2016-08-15/canonical_electron_fluxtube_%04d.png -r 30 -pix_fmt yuv420p \
 -vcodec libx264 -vf "scale=trunc(iw/2)*2:trunc(ih/2)*2" -y ../output/2016-08-15/out.mp4

ffmpeg version 2.8.1 Copyright (c) 2000-2015 the FFmpeg developers
  built with Apple LLVM version 6.0 (clang-600.0.56) (based on LLVM 3.5svn)
  configuration: --prefix=/Users/vonderlinden2/homebrew/Cellar/ffmpeg/2.8.1_1 --enable-shared --enable-pthreads --enable-gpl --enable-version3 --enable-hardcoded-tables --enable-avresample --cc=clang --host-cflags= --host-ldflags= --enable-opencl --enable-libx264 --enable-libmp3lame --enable-libvo-aacenc --enable-libxvid --enable-vda
  libavutil      54. 31.100 / 54. 31.100
  libavcodec     56. 60.100 / 56. 60.100
  libavformat    56. 40.101 / 56. 40.101
  libavdevice    56.  4.100 / 56.  4.100
  libavfilter     5. 40.101 /  5. 40.101
  libavresample   2.  1.  0 /  2.  1.  0
  libswscale      3.  1.101 /  3.  1.101
  libswresample   1.  2.101 /  1.  2.101
  libpostproc    53.  3.100 / 53.  3.100
Input #0, image2, from '../output/2016-08-15/canonical_electron_fluxtube_%04d.png':
  Duration: 00:00:07.67, start: 0.000000, bitrate: N/A
    Stream #0

In [18]:
pwd

u'/Users/vonderlinden2/rsx_analysis/canonical_flux_tubes/source'

In [10]:
visit.SaveWindow?